In [11]:
from etl import create_spark_session

spark = create_spark_session()
spark

In [12]:
sc = spark.sparkContext

config = sc._jsc.hadoopConfiguration()

config.set("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")
sc.setSystemProperty("com.amazonaws.services.s3.enableV4", "true")

input_data = "s3a://udacity-dend/"

output_data = "s3a://udacity-dend/output/"

In [13]:
# get filepath to song data file
song_data = input_data + "song_data/*/*/*/*.json"

# read song data file
df = spark.read.json(song_data)

df.createOrReplaceTempView('song_table')

# extract columns to create songs table
songs_table = spark.sql("""
    SELECT 
    DISTINCT(song_id), title, artist_id, year, duration
    FROM song_table
    WHERE song_id IS NOT NULL
    """)

# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy("year","artist_id").mode("overwrite").parquet(output_data + "songs_table/")

In [7]:
# extract columns to create artists table
artists_table = spark.sql("""
                    SELECT artist_id,
                    artist_name as name, 
                    artist_location as location,
                    artist_latitude as latitude,
                    artist_longitude as longitude
                    FROM song_table
                    """)

# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data + "artists_table/")

In [14]:
input_data = "s3a://udacity-dend-clemens/"

# get filepath to log data file
log_data = input_data + "log_data/*.json"

# read log data file
df = spark.read.json(log_data)

In [15]:
# filter by actions for song plays
df = df.filter(df.page == 'NextSong')

df.createOrReplaceTempView("log_data_table")

users_table = spark.sql("""
    SELECT
    DISTINCT(userId) as user_id,
    firstName as first_name,
    lastName as last_Name,
    gender,
    level
    FROM log_data_table
    WHERE userId IS NOT NULL
    """)


# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data + "users_table/")

"""
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: x/1000)
df = 

# create datetime column from original timestamp column
get_datetime = udf()
df =
"""

Py4JJavaError: An error occurred while calling o142.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [ ]:
# extract columns to create time table
time_table = spark.sql("""
    SELECT timestamp as start_time,
    hour(timestamp) as hour,
    day(timestamp) as day,
    weekofyear(timestamp) as week,
    month(timestamp) as month,
    year(timestamp) as year,
    weekday(timestamp) as weekday
    FROM(
    SELECT to_timestamp(ts/1000) as timestamp
    FROM log_data_table
    WHERE ts IS NOT NULL)
    """)

# write time table to parquet files partitioned by year and month
time_table.write.partitionBy('year','month').mode('overwrite').parquet(output+"time_table/")

In [21]:

# read in song data to use for songplays table
#songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
song_df = spark.sql("""
    SELECT
    monotonically_increasing_id() as songplay_id,
    to_timestamp(ts/1000) as start_time,
    year(to_timestamp(ts/1000)),
    month(to_timestamp(ts/1000)),
    lt.userId as userId,
    lt.level,
    st.song_id,
    st.artist_id,
    lt.sessionId as session_id,
    lt.location,
    lt.userAgent as user_agent
    FROM log_data_table lt
    JOIN song_table st
    ON st.artist_name = lt.artist
    AND lt.song = st.title
    """)
songplays_table.write.partitionBy('year', 'month').mode('overwrite').parquet(output+"songplays_table/")

DataFrame[songplay_id: bigint, start_time: timestamp, year(CAST(to_timestamp((CAST(lt.`ts` AS DOUBLE) / CAST(1000 AS DOUBLE))) AS DATE)): int, month(CAST(to_timestamp((CAST(lt.`ts` AS DOUBLE) / CAST(1000 AS DOUBLE))) AS DATE)): int, userId: string, level: string, song_id: string, artist_id: string, session_id: bigint, location: string, user_agent: string]